# Basic Fourier imaging

In this first tutorial / exercise session, we will explore properties of te Fourier transform: the most crucial piece of math in radio interferometry.

## Jupyter notebooks

For the practical exercises, we will use jupyter notebooks and Python 3.7. If you have never used this before, take some time to explore the menu options. There are text cells like this one, and code cells. To execute a cell, press either `<ctrl-enter>`, `<shift-enter>`, or `<alt-enter>`. Try them all and see what happens.

To edit a cell, click on it or press `<enter>`. To leave cell-editing mode and enter notebook-editing mode, press `<esc>`. To remove a cell, go to notebook-editing mode, select the cell, and press `x`. 

## Harmonic signals / Fourier relations

We begin by deriving/recalling a bunch of relations that will come in handy during the rest of the course. As is customary, we will mostly work in the complex domain, describing for example the electric field at a fixed position in space as

$$
V(t) = \cos(2\pi\nu t) = \Re\left\lbrace \mathrm{e}^{\pm2\pi\mathrm{i}\nu t}\right\rbrace,
$$
where $\nu$ is the frequency and $t$ is time. We call
$$
\mathcal{V}(t) =  \mathrm{e}^{\pm2\pi\mathrm{i}\nu t}
$$
the "complex voltage" to distinguish it from the actual voltage. The choice of sign here is arbitrary, but in any real world applications, whatever sign one chooses by convention at one point in for example a signal chain, determines the signs/conjugations at other points.

For the time-to-frequency Fourier transform, we use the following form:
$$
f(\nu) = \mathcal{F}^-\left\lbrace V(t)\right\rbrace = \int_{-\infty}^{+\infty} V(t) \mathrm{e}^{-2\pi\mathrm{i}\nu t}\ \mathrm{d}t
$$
with inverse
$$
V(t) = \mathcal{F}^+\left\lbrace f(\nu)\right\rbrace = \int_{-\infty}^{+\infty} f(\nu) \mathrm{e}^{+2\pi\mathrm{i}\nu t}\ \mathrm{d}\nu
$$


**Remember that if something seems too simple to be true, it is most likely correct**

# Interactive part

The cell below ensures that we have all the pylab and matplotlib functions available in the main Python namespace. It also ensures that other python modules are reloaded at every cell evaluation. This makes editing complex code elsewhere (for example in fourierexploration.py) , and immediately evaluating it here much easier. One does not have to re-evaluate the entire notebook after every change!


In [1]:
%pylab widget
%load_ext autoreload
%autoreload 2

Populating the interactive namespace from numpy and matplotlib


### 1.5) Sinc interactive

In the cell below we plot this sinc function for a certain bandwidth and delay. Read the code and make sure you understand each step. Play around with the sliders to build some intuition. When does the first null occur? Does that agree with what you expected?

In [2]:
import ipywidgets # Module that allows one to interact with plots

def plot_sinc(ax, bandwidth_plot_range_MHz, delay_us):
    '''
    Plot the sinc function sin(pi delta_nu tau)/(pi delta_nu tau).
    
    Parameters
    ----------
    
    ax : matplotlib.AxesSubplot
        Subplot to draw on.
        
    bandwidth_plot_range_MHz : float
        Maximum bandwidth delta_nu in the plot in MHz.
        
    delay_us : float
        Delay tau in microseconds.
    '''
    dnu_MHz = linspace(0.0, bandwidth_plot_range_MHz, 300)
    result = sinc(dnu_MHz*delay_us)

    ax.cla()
    ax.set_title(r'$\frac{\sin(\pi\Delta\nu\tau)}{\pi\Delta\nu\tau}$ for $\tau$ = %.3f $\mu$s' % delay_us)
    ax.plot(dnu_MHz, result, lw=2)
    ax.set_xlabel(r'Bandwidth $\Delta\nu$ [MHz]')
    ax.set_xlim(dnu_MHz.min(), dnu_MHz.max())
    ax.set_ylim(-0.25,1.03)
    ax.grid()

    
# Create the figure and interact with the "plot_sinc" function
fig_1 = figure(figsize=(6,4), dpi=150)  # Create a figure using matplotlib
ax_1 = fig_1.subplots(1,1)              # Create sub plots (1 row, 1 column)

layout = ipywidgets.Layout(width='80%')
desc_style =  {'description_width': 'initial'}

_ = ipywidgets.interact(
    plot_sinc,
    ax=ipywidgets.fixed(ax_1),
    bandwidth_plot_range_MHz=ipywidgets.FloatSlider(
        min=0.05, max=10, step=0.05, value=1,
        description=r'Plot $\Delta \nu$ [MHz]: ',
        layout=layout,
        readout_format='.3f',
        style=desc_style),
    delay_us=ipywidgets.FloatSlider(
        min=0.000, max=10, step=0.001, value=1,
        description=r'$\tau$ [$\mu$s]: ',
        layout=layout,
        readout_format='.3f',
        style=desc_style))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(FloatSlider(value=1.0, description='Plot $\\Delta \\nu$ [MHz]: ', layout=Layout(width='8…

### 1.6) Heterodyne

Most radio telescope receiver systems involve up- and down converters that shift the frequency of the signals from the sky up or down by multiplying the signal analoguely with a local oscillator. This process is called "heterodyning", and the most common radio receiver chain is the the so-called superheterodyne receiver in which the incoming signal is mixed up or down by a tunable amount, to fit right into a fixed-frequency range amplifier and filtering chain called the intermediate frequency, or "IF". The part that performs the multiplication is the mixer:

![Heterodyne mixer (Wikipedia)](./IdealMixer.png "Heterodyne mixer (Wikipedia).")

We start deriving what happens there using the addition rules for sine and cosine functions. This is easiest starting from the complex domain.

Given angles $a$ and $b$, derive from 
$$ \mathrm{e}^{\mathrm{i}(a+b)}$$
that
$$ \cos(a+b) = \cos a \cos b - \sin a \sin b$$
and
$$ \sin(a+b) = \sin a \cos b + \cos a \sin b.$$

Show that a mixer multiplying two cosine waves (for example input and local oscillator) produces a signal that contains the sum as well as the difference frequency:
$$\cos 2\pi\nu_\mathrm{in}t \cos 2\pi\nu_\mathrm{LO}t  = \frac{1}{2}\left[\cos(2\pi(\nu_\mathrm{in}+\nu_\mathrm{LO}) t) + \cos(2\pi(\nu_\mathrm{in}-\nu_\mathrm{LO}) t)\right]$$

To create a down-converter module, one needs to add a filter that rejects the highest mixing product (heterodyne) and passes the lowest heterodyne.  

Note that when the mixing is done in the complex domain (for example digitally), it simply amounts to applying the Fourier shift theorem.

### ANSWER

$$ \mathrm{e}^{\mathrm{i}(a+b)} = \mathrm{e}^{\mathrm{i}a}\mathrm{e}^{\mathrm{i}b}$$
$$\cos(a+b)+\mathrm{i}\sin(a+b) = (\cos a + \mathrm{i}\sin a)(\cos b + \mathrm{i} \sin b)$$
$$\cos(a+b)+\mathrm{i}\sin(a+b) = \cos a \cos b - \sin a \sin b + \mathrm{i}(\sin a \cos b + \cos a \sin b)$$
Where the real part gives the rule for the cosine and the imaginary part that for the sine.

Now, take the mean of the equations for the cosine of the sum and difference of two angles:
$$\frac{1}{2}(\cos(a+b) + \cos(a-b)) = \frac{1}{2} (2 \cos a \cos b -\sin a \sin b + \sin a \sin b) = \cos a \cos b$$

The cell below allows you to play with mixing different frequencies. Study it carefully, then run it.

In [3]:
def plot_heterodyne(ax, f_1_MHz, f_2_MHz):
    time_s = linspace(0, 200e-9, 1000)
    time_ns = time_s*1e9
    frequency_1_hz = 1e6*f_1_MHz
    frequency_2_hz = 1e6*f_2_MHz

    signal_1 = cos(2*pi*frequency_1_hz*time_s)
    signal_2 = cos(2*pi*frequency_2_hz*time_s)
    signal_sumfreq = 0.5*cos(2*pi*(frequency_1_hz+frequency_2_hz)*time_s)
    signal_difffreq = 0.5*cos(2*pi*(frequency_1_hz-frequency_2_hz)*time_s)
    
    for a in ax:
        a.cla()
        
    ax[0].set_title(f'Input signals at {f_1_MHz:.1f} and {f_2_MHz:.1f} MHz')
    ax[0].plot(time_ns, vstack([signal_1, signal_2]).T)
    ax[0].set_xticklabels([])

    ax[1].set_title('Sum of inputs')
    ax[1].plot(time_ns, signal_1+signal_2, lw=2, alpha=0.5)
    ax[1].set_xticklabels([])

    ax[2].set_title('Product of inputs')
    ax[2].plot(time_ns, signal_1*signal_2, lw=2, alpha=0.5)
    ax[2].set_xticklabels([])

    ax[3].set_title(f'Components in output at: {f_2_MHz-f_1_MHz:.1f} and {f_1_MHz+f_2_MHz:.1f} MHz')
    ax[3].plot(time_ns, vstack([signal_sumfreq, signal_difffreq]).T, lw=2, alpha=0.5)
    ax[3].set_xlabel('Time [ns]')


# Create the figure and axes, and interact
fig_2 = figure(figsize=(8,7), dpi=150)
ax_2 = fig_2.subplots(4,1)

layout = ipywidgets.Layout(width='80%')
_ = ipywidgets.interact(
    plot_heterodyne,
    ax=ipywidgets.fixed(ax_2),
    f_1_MHz=ipywidgets.FloatSlider(min=10., max=300., step=0.2, value=175., layout=layout),
    f_2_MHz=ipywidgets.FloatSlider(min=10., max=300., step=0.2, value=200., layout=layout))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(FloatSlider(value=175.0, description='f_1_MHz', layout=Layout(width='80%'), max=300.0, m…

### 1.7)
See what happens when the input frequencies are close together and far apart. Explain what you see.

### ANSWER

Well, you *have* proven that the *sum* of two waves can be written as the product of two others...

# In two dimensions

The relations derived above are easily extended to two dimensions. Because a radio interferometer does not directly measure the sky, but instead samples (to first order, and under certain conditions) the two-dimensional Fourier transform of the sky, it is time to gain some intuition in that respect. The Fourier image is called the "uv-plane" after the conventional names of its coordinates.

The fourierexploration.py module contains the gui and simulation code for the next cell. Feel free to have a look at it, but right now it is more important to start playing with it.

The original image is on the top left, its Fourier transform on the top right, the modified uv-plane bottom right, and the Fourier transform of the modified uv-plane ("Dirty image") on the bottom left. The uv-planes indicate amplitude by brightness, and phase by color. In the uv-plane, large scales end up at the centre, while small scales end up at the edges. Execute the next cell and answer the questions below it.

In [4]:
import fourierexploration as fe

fe_gui = fe.FourierExplorerGui(figsize=(8,8), dpi=150)

shirt = fe.imread('shirt-bw.png')
pikachu = fe.imread('pikachu.png')
poliwhirl = fe.imread('poliwhirl.png')

fe_gui.interact({'Point source': fe.point_source_image,
                 'Pikachu': pikachu,
                 'Shirt': shirt,
                 'Poliwhirl': poliwhirl})

ImportError: cannot import name '_fftpack' from 'scipy.fftpack' (/home/brentjens/VirtualEnvs/jupyter/lib/python3.8/site-packages/scipy/fftpack/__init__.py)

### 1.8) Point source

Start with the point source. It should be at the centre. Move the shift sliders, and describe what happens. Now try the maximum and minimum baseline sliders. Describe what you find.

### 1.9) Pikachu

Set everything back to the original positions and load Pikachu. Try both the dB and Linear brightness scales for the uv-plane. Do you see a sinc-like pattern? What does that tell you about the drawing? Verify your hypothesis by using the sliders.

### 1.10) Shirt

The uv-plane is highly structured. Why? Try to associate parts of the uv-plane with features in the image. Use the zoom-button (second from right in the button-bar) if necessary. What disappears first if you increase the minimum baseline? What happens when you reduce the maximum baseline to that value and set the minimum baseline to 0? What causes the bright spots far in the dark corners of the uv-plane?

# APSYNSIM

There is an interesting tool written by Ivan Marti-Vidal (Onsala, SE), in which one can do similar experiments with actual uv-coverages of real radio telescopes. Executing the next cell will start the program.

In [5]:
%%capture
! python3 ./apsynsim/SCRIPT/APSYNSIM_Python3.py